In [ ]:
#importing libraries
library(tidyverse)
library(tidymodels)
library(repr)
library(RColorBrewer)
library(dplyr) 
install.packages("kknn")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [ ]:
diabetes_data <- read_csv("https://raw.githubusercontent.com/altonho67/dsci-100-2023w1-group-31/main/data/diabetes_2.csv")
diabetes_data
set.seed(2020)
diabetes_data <- diabetes_data %>%
                 mutate(Outcome = as.factor(Outcome))|>
                 mutate(Outcome = fct_recode(Outcome, "Diabetes" = "1", "None" = "0")) |>
                 mutate_at(c('SkinThickness','Insulin','BloodPressure','Glucose'), ~na_if(., 0))
                 
diabetes_data

diabetes_split <- initial_split(diabetes_data, prop = 3/4, strata = Outcome)
diabetes_training <- training(diabetes_split)
diabetes_testing <- testing(diabetes_split)


#diabetes_vfold <- vfold_cv(diabetes_training, v = 10, strata = Outcome)
#diabetes_vfold

lm_spec<-linear_reg() |>
  set_engine("lm") |>
  set_mode("regression")

diabetes_recipe <- recipe(Glucose ~ BMI +Age, 
                        data = diabetes_training) 
  

# use the BMI and Age to predict the glucose
diabetes_fit <- workflow() |>
  add_recipe(diabetes_recipe) |>
  add_model(lm_spec) |>
  fit(data = diabetes_training)

diabetes_fit
#this one shows the linear relationship between the BMI and Age with the Glucose, this may not be useful, and can be deleted if you want

diabetes_split <- initial_split(diabetes_data, prop = 3/4, strata = Outcome)
diabetes_training <- training(diabetes_split)
diabetes_testing <- testing(diabetes_split)

predict_outcome_spec<-nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
  set_engine("kknn") |>
  set_mode("classification")

predict_recipe <- recipe(Outcome~ Glucose + BMI +Age,data=diabetes_training) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

knn_fit <- workflow() |>
  add_recipe(predict_recipe) |>
  add_model(predict_outcome_spec) |>
  fit(data = diabetes_training )

diabetes_outcome_predictions <- predict(knn_fit, diabetes_testing)|>
    bind_cols(diabetes_testing)


Diabetes_outcome_predictions
#this is the knn neighbor prediction of DIabetes outcome using BMI, Glucose and Age, as these three show a positive relationship with diabetes outcome in the proposal, so I am using them to approximate the outcome. More can be added, such as choosing the best k for knn spec and use vfold.
